In [1]:
!pip install ultralytics streamlit pyngrok roboflow opencv-python-headless -q
from google.colab import drive
drive.mount('/content/drive')
!rm -rf mask-hairnet-gloves-iopkp-1
from roboflow import Roboflow
rf = Roboflow(api_key="vLBWuwn1rsPb9kU40N44")
project = rf.workspace("workshopr").project("mask-hairnet-gloves-iopkp")
version = project.version(1)
dataset = version.download("yolov11")
DATASET_DIR = dataset.location
DATA_YAML_PATH = f"{DATASET_DIR}/data.yaml"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 142.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 102.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 111.8 MB/s eta 0:00:00
Mounted at /content/drive
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to mask-hairnet-gloves-1 in yolov11:: 100%|██████████| 870/870 [00:00<00:00, 7397.17it/s]


In [22]:
from ultralytics import YOLO
import torch

# 1. Load the pre-trained nano model (it's fast and good for PPE)
model = YOLO('yolo11n.pt')

# 2. Start Training with more epochs
# We use 100 epochs, but it might stop early if it stops improving
results = model.train(
    task='detect',
    data='/content/mask-hairnet-gloves-1/data.yaml', # Ensure this path is correct!
    epochs=100,      # Increased from 10
    imgsz=640,       # Standard size for YOLOv11
    batch=16,        # Adjust based on Colab GPU memory
    patience=20,     # Early stopping
    save=True,
    name='ppe_yolo11_high_epochs'
)

# 3. Backup the NEW best weights
MODEL_SAVE_PATH = '/content/runs/detect/ppe_yolo11_high_epochs/weights/best.pt'
!cp {MODEL_SAVE_PATH} /content/best_ppe_yolo11_model.pt

print(f"\n✅ Training Complete! New model saved to: /content/best_ppe_yolo11_model.pt")

Ultralytics 8.3.240 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/mask-hairnet-gloves-1/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=ppe_yolo11_high_epochs2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=20, persp

In [23]:
%%writefile app.py
import streamlit as st
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator, colors
import cv2
import tempfile
import numpy as np
import yaml

# --- Updated Path to match your actual folder ---
MODEL_PATH = 'best_ppe_yolo11_model.pt'
DATA_YAML_PATH = '/content/mask-hairnet-gloves-1/data.yaml'

@st.cache_resource
def load_model():
    return YOLO(MODEL_PATH)

MODEL = load_model()
# Get class names directly from the model weights for 100% accuracy
NAMES = MODEL.names

st.set_page_config(page_title="PPE Detector", layout="wide")
st.title("🛡️ PPE Object Detector")

# --- Sidebar ---
st.sidebar.header("Settings")
confidence = st.sidebar.slider("Confidence Threshold", 0.01, 1.0, 0.20)

st.sidebar.markdown("---")
st.sidebar.subheader("Model Class Mapping")
# This will show you exactly what index the model uses for 'glove' or 'hairnet'
st.sidebar.json(NAMES)

count_display = st.sidebar.empty()

def process_frame(frame, conf_threshold):
    results = MODEL.predict(frame, conf=conf_threshold, verbose=False)[0]
    annotator = Annotator(frame.copy(), line_width=2)
    counts = {name: 0 for name in NAMES.values()}

    if results.boxes:
        for box in results.boxes:
            c = int(box.cls[0])
            label = f"{NAMES[c]} {box.conf[0]:.2f}"
            annotator.box_label(box.xyxy[0], label, color=colors(c, True))
            counts[NAMES[c]] += 1

    return annotator.result(), counts

# --- Main Logic ---
source_option = st.sidebar.radio("Select Source", ('Image Upload', 'Video Upload'))

if source_option == 'Image Upload':
    uploaded_file = st.file_uploader("Upload Image", type=['jpg', 'jpeg', 'png'])
    if uploaded_file:
        img = cv2.imdecode(np.frombuffer(uploaded_file.read(), np.uint8), 1)
        res_img, counts = process_frame(img, confidence)
        st.image(res_img, channels="BGR", use_container_width=True)
        with count_display.container():
            st.write("### 📊 Detection Counts")
            for k, v in counts.items(): st.write(f"**{k}:** `{v}`")

elif source_option == 'Video Upload':
    uploaded_file = st.file_uploader("Upload Video", type=['mp4', 'avi'])
    if uploaded_file:
        tfile = tempfile.NamedTemporaryFile(delete=False)
        tfile.write(uploaded_file.read())
        cap = cv2.VideoCapture(tfile.name)
        st_frame = st.empty()
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret: break
            res_frame, counts = process_frame(frame, confidence)
            st_frame.image(res_frame, channels="BGR", use_container_width=True)
            with count_display.container():
                st.write("### 📊 Detection Counts")
                for k, v in counts.items(): st.write(f"**{k}:** `{v}`")
        cap.release()

Overwriting app.py


In [24]:
from ultralytics import YOLO
import os

# 1. Load the model
test_model = YOLO('/content/best_ppe_yolo11_model.pt')

# 2. Path we found earlier
TEST_PATH = '/content/mask-hairnet-gloves-1/test/images'

# 3. Predict with very low confidence to catch faint objects
results = test_model.predict(source=TEST_PATH, save=True, conf=0.1, imgsz=640)

print("\n--- TEST IMAGE RESULTS ---")
for r in results[:3]:
    labels = [test_model.names[int(c)] for c in r.boxes.cls]
    print(f"File: {os.path.basename(r.path)} | Detected: {labels}")


image 1/18 /content/mask-hairnet-gloves-1/test/images/WIN_20241230_16_34_22_Pro_jpg.rf.d2d4d30700fd33aac017dead4546d95f.jpg: 384x640 1 hairnet, 1 mask, 10.5ms
image 2/18 /content/mask-hairnet-gloves-1/test/images/WIN_20241230_16_34_32_Pro_jpg.rf.cf7c8c824a9eb1ae272347933c42d780.jpg: 384x640 1 hairnet, 1 mask, 8.7ms
image 3/18 /content/mask-hairnet-gloves-1/test/images/WIN_20241230_16_34_48_Pro-2-_jpg.rf.3cfc15a7ff17a9dbf8951155fa3b7349.jpg: 384x640 1 hairnet, 1 mask, 8.0ms
image 4/18 /content/mask-hairnet-gloves-1/test/images/WIN_20241230_16_34_49_Pro_jpg.rf.7b2d240da743c7d63f651437439c1dfb.jpg: 384x640 1 hairnet, 1 mask, 8.5ms
image 5/18 /content/mask-hairnet-gloves-1/test/images/WIN_20241230_16_35_14_Pro-2-_jpg.rf.908968c3d892122b6eb7e8e24ba06a88.jpg: 384x640 1 mask, 8.3ms
image 6/18 /content/mask-hairnet-gloves-1/test/images/WIN_20241230_16_35_41_Pro_jpg.rf.be669ed7ae9176493e70b6150f853f5a.jpg: 384x640 1 mask, 8.9ms
image 7/18 /content/mask-hairnet-gloves-1/test/images/WIN_20241230

In [25]:
# 1. Set your Auth Token (Only needs to be done once)
!ngrok config add-authtoken 36mTJozBS7Ott9CjNVbSphx2yJy_26mhNGXXtL27KMDD822cP

# 2. Make sure the model is in the right spot
!cp /content/runs/detect/ppe_yolo11_detect_v1/weights/best.pt /content/best_ppe_yolo11_model.pt

# 3. Import and Kill old sessions
from pyngrok import ngrok
import subprocess
import time

try:
    ngrok.kill()
except:
    pass

# 4. Start Streamlit
print("🚀 Starting Streamlit...")
proc = subprocess.Popen(["streamlit", "run", "app.py"])
time.sleep(10) # Give it time to boot

# 5. Create Tunnel
try:
    public_url = ngrok.connect(8501)
    print(f"\n✅ SUCCESS!")
    print(f"🔗 Click here to open your dashboard: {public_url}")
    print("\nKeep this cell running while you use the app.")
except Exception as e:
    print(f"❌ Error: {e}")

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
🚀 Starting Streamlit...

✅ SUCCESS!
🔗 Click here to open your dashboard: NgrokTunnel: "https://postmesenteric-ultramicroscopical-shin.ngrok-free.dev" -> "http://localhost:8501"

Keep this cell running while you use the app.


In [26]:
# Run this to make sure your best model is backed up
!cp /content/best_ppe_yolo11_model.pt /content/drive/MyDrive/Final_PPE_YOLOv11_Project.pt
print("✅ Model weights backed up to Google Drive!")

✅ Model weights backed up to Google Drive!


In [27]:
%%writefile requirements.txt
ultralytics
streamlit
opencv-python-headless
pyngrok
pyyaml
numpy

Writing requirements.txt
